In [ ]:

import math
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

ogtest = pd.read_csv("Freyja_Pumpkins.csv")
ogtrain = pd.read_csv("Gotem Pumpkins.csv")

test = ogtest.copy()
train = ogtrain.copy()

def bin(type):
    if type == "Çerçevelik":
        return 1
    else: 
        return 0
X_train = np.array([list(value) for value in train.drop("Class", axis=1).values]) 

Y_train = np.array([val for val in map(bin, list(train['Class']))])
testdata= np.array([list(value) for value in test.drop("Class", axis=1).values]) 
testclass = np.array([val for val in map(bin, list(test['Class']))])


DEFINING FUNCTIONS FOR SIGMOID, LOGISTIC LOSS FUNCTION, GRADIENT DESCENT AMD Z SCORE NORMALISATION,

In [57]:
def sigmoid(z):

    g = 1/(1+np.exp(-z))
   
    return g



def compute_cost_logistic(X, y, w, b):
    m,n= X.shape
    total_cost = 0.
    for i in range(m):
        z = np.dot(X[i],w) + b
        f_wb_i = sigmoid(z)
        loss =  -y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i)
        total_cost+=loss    
    total_cost /= m
    return total_cost

def compute_gradient_logistic(X, y, w, b): 
   
    m,n = X.shape
    dj_dw = np.zeros((n,))                          
    dj_db = 0.

    for i in range(m):
        z=np.dot(X[i],w)+b
        f_wb_i = sigmoid(z)         
        err_i = f_wb_i - y[i]                       
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i,j]      
        dj_db = dj_db + err_i
    dj_dw = dj_dw/m                                   
    dj_db = dj_db/m                                   
        
    return dj_db, dj_dw
def gradient_descent(X, y, w_in, b_in, alpha, num_iters): 

    J_history = []
    w = w_in.copy()  
    b = b_in
    
    for i in range(num_iters):
    
        dj_db, dj_dw = compute_gradient_logistic(X, y, w, b)   
        w = w - alpha * dj_dw               
        b = b - alpha * dj_db               
        if i<100000:      # prevent resource exhaustion 
            J_history.append( compute_cost_logistic(X, y, w, b) )

        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]}   ")
        
    return w, b, J_history 

def znormal(X):
    mean    = np.mean(X, axis=0)                 
    stdev  = np.std(X, axis=0)+0.00001                 
    X_norm = (X - mean) / stdev 
    return (X_norm, mean, stdev)

RUNNING THROUGH REQUIRED NUMBER OF ITERATIONS TO COST FUNCTION

In [58]:
w = np.zeros_like(X_train[0])
b  = 0.
alpha = 0.05
iterations = 10000
w_i, b_i, J_hist = gradient_descent(znormal(X_train)[0],Y_train, w, b, alpha, iterations)


Iteration    0: Cost 0.6602238443152165   
Iteration 1000: Cost 0.3250058919748191   
Iteration 2000: Cost 0.32410117684397327   
Iteration 3000: Cost 0.32328121262770104   
Iteration 4000: Cost 0.32251498927033545   
Iteration 5000: Cost 0.321789993097854   
Iteration 6000: Cost 0.32109856492999156   
Iteration 7000: Cost 0.320435854031846   
Iteration 8000: Cost 0.3197986558211524   
Iteration 9000: Cost 0.3191847414699596   


RUNNING THE LOGISTIC REGRESSION MODEL ON TEST DATASET AND ARRIVING AT THE PREDICTIONS

In [73]:

normalizedtestdata=znormal(testdata)
pred = sigmoid(np.dot(normalizedtestdata[0], w_i) + b_i)

CALCULATING CONFUSION MATRIX PARAMETERS TO CALCULATE ACCURACY OF THE MODEL

In [93]:
binpred = [1 if value >= 0.5 else 0 for value in pred]
TruePos = 0
TrueNeg = 0
FalsePos = 0
FalseNeg =  0
for index, predicted in enumerate(binpred):
    actual = testclass[index]
    if predicted > actual:
        FalsePos += 1
    elif actual> predicted:
        FalseNeg += 1
    elif actual  == predicted:
        if actual == 0: 
            TrueNeg += 1 
        else: 
            TruePos += 1
print(f"accuracy of the model is: {(TruePos + TrueNeg)*100/len(binpred)}%")

accuracy of the model is: 88.4%
